In [22]:
#Dependencies
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [23]:
#read csv into df
df = pd.read_csv("Data/Levels_Fyi_Salary_Data.csv")
# columns to drop... salary_currency, salary, work_year
df.sample(25)

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
1160,9/7/2018 10:23:34,Qualcomm,Staff Engineer,Software Engineer,184000,"San Diego, CA",8.0,6.0,asic,140000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
55697,6/26/2021 10:46:16,Nvidia,IC4,Hardware Engineer,251000,"Boston, MA",9.0,0.0,Verification,180000.0,...,0,0,0,1,0,0,0,0,Asian,Master's Degree
22655,6/16/2020 19:19:29,Epic Systems,SDE,Software Engineer,105000,"Madison, WI",1.0,1.0,Full Stack,100000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
55087,6/21/2021 10:27:10,Chevron,PSG 20,Software Engineer,121000,"Houston, TX",3.0,3.0,Full Stack,109000.0,...,0,0,0,1,0,0,0,0,Asian,Bachelor's Degree
41348,2/5/2021 21:36:31,Uber,Sr Eng Manager,Software Engineering Manager,650000,"San Francisco, CA",14.0,4.0,Full Stack,230000.0,...,0,0,0,0,0,0,0,0,NaN,Bachelor's Degree
60288,8/5/2021 0:33:29,Qualcomm,Senior Staff Engineer,Hardware Engineer,185000,"Hsin-chu, TP, Taiwan",17.0,13.0,Embedded Systems,110000.0,...,0,0,0,0,0,0,0,0,NaN,Master's Degree
36677,12/6/2020 15:38:10,Facebook,IC3,Product Designer,149000,"Menlo Park, CA",1.0,0.0,User Experience (UX),105000.0,...,0,0,0,0,0,0,0,0,NaN,Bachelor's Degree
38043,12/23/2020 21:38:07,Facebook,IC5,Product Designer,295000,"Menlo Park, CA",10.0,0.0,User Experience (UX),187000.0,...,0,0,0,1,0,0,0,0,Asian,Bachelor's Degree
33712,10/25/2020 14:41:33,Amazon,L4,Software Engineer,147000,"Seattle, WA",0.0,0.0,Full Stack,112000.0,...,0,0,0,1,0,0,0,0,Asian,Master's Degree
50177,5/1/2021 15:11:22,Apple,ICT4,Software Engineer,330000,"Cupertino, CA",11.0,5.0,Firmware,188000.0,...,0,0,0,1,0,0,0,0,Asian,Master's Degree


In [24]:
df.nunique()


timestamp                  62561
company                     1631
level                       2923
title                         15
totalyearlycompensation      893
location                    1050
yearsofexperience             65
yearsatcompany                81
tag                         3058
basesalary                   482
stockgrantvalue              612
bonus                        335
gender                         4
otherdetails               12841
cityid                      1045
dmaid                        149
rowNumber                  62642
Masters_Degree                 2
Bachelors_Degree               2
Doctorate_Degree               2
Highschool                     2
Some_College                   2
Race_Asian                     2
Race_White                     2
Race_Two_Or_More               2
Race_Black                     2
Race_Hispanic                  2
Race                           5
Education                      5
dtype: int64

In [25]:
titles_df = df[df["title"]=="Data Scientist"]
titles_df


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
419,6/5/2018 14:06:30,LinkedIn,Senior,Data Scientist,233000,"San Francisco, CA",4.0,0.0,Data Analysis,162000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
440,6/8/2018 9:49:25,Microsoft,64,Data Scientist,218000,"Seattle, WA",11.0,11.0,ML / AI,165000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
444,6/8/2018 17:55:09,ebay,26,Data Scientist,180000,"San Jose, CA",10.0,5.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
454,6/10/2018 19:39:35,Twitter,Staff,Data Scientist,500000,"San Francisco, CA",4.0,4.0,ML / AI,200000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
495,6/17/2018 11:39:38,Facebook,5,Data Scientist,370000,"Seattle, WA",8.0,3.0,NaN,190000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62240,3/26/2019 17:25:02,IBM,8,Data Scientist,155000,"Washington, DC",5.0,2.0,ML / AI,141000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62283,6/17/2018 21:07:18,Facebook,E3,Data Scientist,150000,"Singapore, SG, Singapore",0.0,0.0,ML / AI,150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62285,7/16/2018 20:11:40,ADP,Sr UX Researcher 4,Data Scientist,185000,"Roseland, NJ",8.0,2.0,UX Research,150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62529,6/12/2018 20:54:06,Google,T6,Data Scientist,685000,"Kirkland, WA",22.0,2.0,ML / AI,221000.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [26]:
# df1 = titles_df.copy()
# filtered_columns = ["company", "location", "yearsofexperience", "totalyearlycompensation",
#                  "Masters_Degree", "Bachelors_Degree"]
# df2 = df1[filtered_columns]
# # print(df1.shape)
# df2.head()

In [27]:
clean_df= titles_df.drop(columns=['timestamp', 'level', 'yearsatcompany', 'otherdetails','Highschool', 'Some_College', 
                               'Race_Asian', 'Race_White', 'Race_Two_Or_More', 'Race_Black',"gender",
                              'Race_Hispanic', 'Race', 'Education','cityid','dmaid','rowNumber','tag','bonus','stockgrantvalue','basesalary','title'])
clean_df.head()
#get_dummies/oneHotEncoder on company size and employment type

,company,totalyearlycompensation,location,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",4.0,0,0,0
440,Microsoft,218000,"Seattle, WA",11.0,0,0,0
444,ebay,180000,"San Jose, CA",10.0,0,0,0
454,Twitter,500000,"San Francisco, CA",4.0,0,0,0
495,Facebook,370000,"Seattle, WA",8.0,0,0,0


In [28]:
import re
form_state_one = r'(\w+\s)*(\w+),\s([A-Z]{2})$'
matches_form_one = clean_df.location.str.contains(form_state_one, flags=re.IGNORECASE, na=False)
no_form_one = clean_df.location.str.contains(form_state_one, flags=re.IGNORECASE, na=False).sum()
print(no_form_one)

form_state_two = r'(\w+\s)*(\w+),\s([A-Z]{2}),\s(\w+)'
matches_form_two = clean_df.location.str.contains(form_state_two, flags=re.IGNORECASE, na=False)
no_form_two = clean_df.location.str.contains(form_state_two, flags=re.IGNORECASE, na=False).sum()
print(no_form_two)


clean_df.location[~matches_form_one & ~matches_form_two]

2226
351


C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.
C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


9061    Tel Aviv, Israel
Name: location, dtype: object

In [29]:
clean_df.head()

,company,totalyearlycompensation,location,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",4.0,0,0,0
440,Microsoft,218000,"Seattle, WA",11.0,0,0,0
444,ebay,180000,"San Jose, CA",10.0,0,0,0
454,Twitter,500000,"San Francisco, CA",4.0,0,0,0
495,Facebook,370000,"Seattle, WA",8.0,0,0,0


In [30]:
def US_location_to_state(city):
    # if input is of the form $###,###,###
    if re.match(form_state_one, city, flags=re.IGNORECASE):

        # remove city name and commas
        state = re.sub('(\w+\s)*(\w+),\s','', city)
        
        # return state
        return state
    # otherwise, return NaN
    else:
        return np.nan
    

def location_to_city(city):
    # if input is of the form $###,###,###
    if re.match(form_state_one, city, flags=re.IGNORECASE):

        # remove city name and commas
        city = re.sub(',\s([A-Z]{2})$','', city)
        
        # return state
        return city
    # otherwise, return NaN
    else:
        return np.nan
clean_df = clean_df.loc[clean_df.location != "Tel Aviv, Israel"]
clean_df


,company,totalyearlycompensation,location,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",4.0,0,0,0
440,Microsoft,218000,"Seattle, WA",11.0,0,0,0
444,ebay,180000,"San Jose, CA",10.0,0,0,0
454,Twitter,500000,"San Francisco, CA",4.0,0,0,0
495,Facebook,370000,"Seattle, WA",8.0,0,0,0
...,...,...,...,...,...,...,...
62240,IBM,155000,"Washington, DC",5.0,0,0,0
62283,Facebook,150000,"Singapore, SG, Singapore",0.0,0,0,0
62285,ADP,185000,"Roseland, NJ",8.0,0,0,0
62529,Google,685000,"Kirkland, WA",22.0,0,0,0


In [31]:
clean_df['state'] = clean_df.location.str.extract(f'({form_state_one}|{form_state_two})', flags=re.IGNORECASE)[0].apply(US_location_to_state)

In [32]:
clean_df = clean_df[clean_df['state'].notna()]
clean_df.head()

,company,totalyearlycompensation,location,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state
419,LinkedIn,233000,"San Francisco, CA",4.0,0,0,0,CA
440,Microsoft,218000,"Seattle, WA",11.0,0,0,0,WA
444,ebay,180000,"San Jose, CA",10.0,0,0,0,CA
454,Twitter,500000,"San Francisco, CA",4.0,0,0,0,CA
495,Facebook,370000,"Seattle, WA",8.0,0,0,0,WA


In [33]:
clean_df['city'] = clean_df.location.str.extract(f'({form_state_one})', flags=re.IGNORECASE)[0].apply(location_to_city)
clean_df.drop('location', axis=1, inplace=True)
clean_df.head()

C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,company,totalyearlycompensation,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state,city
419,LinkedIn,233000,4.0,0,0,0,CA,San Francisco
440,Microsoft,218000,11.0,0,0,0,WA,Seattle
444,ebay,180000,10.0,0,0,0,CA,San Jose
454,Twitter,500000,4.0,0,0,0,CA,San Francisco
495,Facebook,370000,8.0,0,0,0,WA,Seattle


In [35]:
print(clean_df["company"].value_counts().head(10))

Amazon          224
Microsoft       202
Facebook        175
Google           83
Apple            75
Uber             60
IBM              58
Capital One      44
LinkedIn         34
Walmart Labs     33
Name: company, dtype: int64


In [36]:
# #Binning he companies- .cut with 4 bins...do we want to specifically design these?
# bin_labels = ['Small', 'Medium', 'Large']
# bins3 = clean_df['company'].value_counts()
# company_bins = pd.cut(bins3, labels=bin_labels).to_list()#does this solve the issue of imbalanced data?
# company_bins

In [37]:
#Binning he companies- .cut with 4 bins...do we want to specifically design these?
bin_labels = ['Small', 'Medium', 'Large', 'Very Large']
bins4 = clean_df['company'].value_counts()
company_bins = pd.cut(bins4, bins=4, labels=bin_labels).to_list()#does this solve the issue of imbalanced data?
company_bins


['Very Large',
 'Very Large',
 'Very Large',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 

In [38]:
#change company name to bin name
companies = []
for company in clean_df['company']:
    if company == "Amazon" or company == "Microsoft" or company == "Facebook":
        company = "Large"
    elif company == "Google" or company == "Apple" or company == "IBM" or company=="Uber":
        company = "Medium"
    else:
        company = "Small"
    companies.append(company)

In [39]:
#reassign company column to bin name
clean_bins_df = clean_df.copy()
clean_bins_df['company'] = companies
clean_bins_df.head(10)
# clean_bins_df['gender'].value_counts()

,company,totalyearlycompensation,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state,city
419,Small,233000,4.0,0,0,0,CA,San Francisco
440,Large,218000,11.0,0,0,0,WA,Seattle
444,Small,180000,10.0,0,0,0,CA,San Jose
454,Small,500000,4.0,0,0,0,CA,San Francisco
495,Large,370000,8.0,0,0,0,WA,Seattle
499,Large,200000,3.0,0,0,0,WA,Seattle
509,Large,340000,11.0,0,0,0,WA,Bellevue
510,Medium,690000,10.0,0,0,0,WA,Kirkland
513,Small,600000,3.0,0,0,0,CA,Los Gatos
523,Small,168000,8.0,0,0,0,CA,Palo Alto


In [41]:
clean_bins_df.to_csv("Cleaned_dataset.csv", sep=',', header=True, encoding='utf-8', index=False)

In [19]:
# #is there linearity between location numerical representation and salary?
# clean_bins2 = clean_bins_df.drop(['location'], axis = 1)
# locations = clean_bins_df['location']
# clean_bins2.head()

In [ ]:
# #Create label Encoder and encode categorical
# from sklearn.preprocessing import LabelEncoder
# lb_make = LabelEncoder()
# clean_bins2['company']= lb_make.fit_transform(clean_bins2['company'])
# clean_bins2['gender']= lb_make.fit_transform(clean_bins2['gender'])

# clean_bins2.head()

In [ ]:
# clean_bins2['gender'].value_counts() #what do we do with 4 genders?